In [1]:
import pandas as pd
from tqdm import tqdm
import nltk.data
import re
import collections

tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')

In [60]:
# with open('../Scraped data/bd_news_dt.json') as infile:
#     data = json.load(infile)

df = pd.read_json('Data/DT/bd_news_dt.json')

In [61]:
df.head()

,_id,is_negative,news_crawled_date,news_headline,news_image_urls,news_keywords,news_location,news_ml_tags,news_naive_tags,news_ner_tags,news_original_tags,news_publish_date,news_reporters,news_text,news_url,newspaper_name,newspaper_url
0,{'$oid': '575f79f55388c1567baebb2a'},True,{'$date': '2016-06-13T23:28:50.269Z'},Ashram residents remain in panic,[http://www.dhakatribune.com/sites/default/fil...,"[bangladesh, ashram, monastery, nityanando, mo...",Pabna,NaN,[crime],"{'dates': ['Friday', 'Saturday', 'October', 'S...",[crime],{'$date': '2016-06-14T01:28:00.000Z'},[[Md Emroz Khandakar]],People who live in Sri Sri Thakur Anukulchandr...,http://www.dhakatribune.com/crime/2016/jun/14/...,Dhaka Tribune,http://www.dhakatribune.com
1,{'$oid': '575f79f85388c1567baebb2b'},True,{'$date': '2016-06-13T23:28:54.437Z'},Bangladeshi abducted from Bandarban border,[http://www.dhakatribune.com/sites/default/fil...,"[naikkangchhari, area, bandarban, woodcutters,...",Bandarban,NaN,[crime],"{'dates': ['Sunday', 'April 15.', 'April 18.']...",[bangladesh],{'$date': '2016-06-14T01:27:00.000Z'},[[S Bashu Das]],Separatists in Myanmar abducted a Bangladeshi ...,http://www.dhakatribune.com/bangladesh/2016/ju...,Dhaka Tribune,http://www.dhakatribune.com
2,{'$oid': '575f79fc5388c1567baebb2c'},True,{'$date': '2016-06-13T23:28:57.408Z'},‘Narayanganj killing was pre-planned’,[http://www.dhakatribune.com/sites/default/fil...,"[murder, rab, picked, preplanned, vehicles, ki...",Narayanganj,NaN,[crime],"{'dates': ['June 20', 'April', 'early hours of...",[crime],{'$date': '2016-06-14T01:25:00.000Z'},[[Tanveer Hossain]],Two personnel of the Rapid Action Battalion ye...,http://www.dhakatribune.com/crime/2016/jun/14/...,Dhaka Tribune,http://www.dhakatribune.com
3,{'$oid': '575f79fe5388c1567baebb2d'},True,{'$date': '2016-06-13T23:29:00.538Z'},Slum kids see no future without birth registra...,[http://www.dhakatribune.com/sites/default/fil...,"[dhaka, school, kids, pay, slum, shabnam, pall...",National,NaN,[accident],"{'dates': [], 'persons_unique': ['Shabnam'], '...",[bangladesh],{'$date': '2016-06-14T01:24:00.000Z'},[Mohammad Jamil Khan],Thirteen-year-old Shabnam had big dreams for h...,http://www.dhakatribune.com/bangladesh/2016/ju...,Dhaka Tribune,http://www.dhakatribune.com
4,{'$oid': '575f7a015388c1567baebb2e'},True,{'$date': '2016-06-13T23:29:03.326Z'},Mobile-based agro advisory service makes farme...,[http://www.dhakatribune.com/sites/default/fil...,"[advisory, life, grameenphone, service, farmer...",National,NaN,[accident],"{'dates': ['December last year'], 'persons_uni...",[bangladesh],{'$date': '2016-06-14T01:22:00.000Z'},[Abu Bakar Siddique],Amela Begum in Rangpur believes she could have...,http://www.dhakatribune.com/bangladesh/2016/ju...,Dhaka Tribune,http://www.dhakatribune.com


In [85]:
def name_disambiguation(names):
    disambiguated_names = []
    for name in names:
        if (name == "Khaleda" or name == "Zia" or name == "Begum Khaleda Zia" or name == "Khaleda Zia’s"):
            disambiguated_names.append("Khaleda Zia")
        elif (name == "Hasina" or name == "Sheikh" or name == "Sheikh Hasina."):
            disambiguated_names.append("Sheikh Hasina")
        elif (name == "Fakhrul"):
            disambiguated_names.append("Mirza Fakhrul Islam Alamgir")
        elif (name == "Muhith" or name == "AMA Muhith" or name == "MA Muhith"):
            disambiguated_names.append("Abul Maal Abdul Muhith")
        elif (name == "Nizami" or name == "Motiur Rahman"):
            disambiguated_names.append("Motiur Rahman Nizami")
        elif (name == "Modi"):
            disambiguated_names.append("Narendra Modi")
        elif (name == "Bangabandhu" or name == "Sheikh Mujib" or name == "Sheikh Mujib" or name == "Bangabandhu Sheikh Mujibur" or name == "Sheikh Mujibur Rahman" or name == "Mujibur Rahman"):
            disambiguated_names.append("Bangabandhu Sheikh Mujibur Rahman")
        elif (name == "Tarique"):
            disambiguated_names.append("Tarique Rahman")
        elif (name == "Avijit" or name == "Roy"):
            disambiguated_names.append("Avijit Roy")
        elif (name == "Mozena"):
            disambiguated_names.append("Dan Mozena")
        elif (name == "Yunus" or name == "Mohammad Yunus"):
            disambiguated_names.append("Muhammad Yunus")
        elif (name == "Obama"):
            disambiguated_names.append("Barack Obama")
        elif (name == "Trump"):
            disambiguated_names.append("Donald Trump")
        elif (name == "Tagore" or name == "Rabindranath" or name == "Robindronath" or name == "Rabindronath" or name == "Robindranath"):
            disambiguated_names.append("Rabindranath Tagore")
        elif (name == "Karzai"):
            disambiguated_names.append("Hamid Karzai")
        elif (name == "Zardari"):
            disambiguated_names.append("Asif Ali Zardari")
        elif (name == "Quader Molla"):
            disambiguated_names.append("Abdul Quader Molla")
        else:
            disambiguated_names.append(name)
    return (list(set(disambiguated_names)))

In [81]:
def org_disambiguation(orgs):
    disambiguated_orgs = []
    for org in orgs:
        if (org == "AL" or org == "the Awami League"):
            disambiguated_orgs.append("Awami League")
        elif (org == "EU" or org == "the European Union"):
            disambiguated_orgs.append("European Union")
        elif (org == "BCL" or org == "Chhatra League"):
            disambiguated_orgs.append("Bangladesh Chhatra League")
        elif (org == "DU"):
            disambiguated_orgs.append("Dhaka University")
        elif (org == "the High Court" or org == "The High Court"):
            disambiguated_orgs.append("High Court")
        elif (org == "the World Bank"):
            disambiguated_orgs.append("World Bank")
        elif (org == "the Supreme Court"):
            disambiguated_orgs.append("Supreme Court")
        elif (org == "UN" or org == "the United Nations"):
            disambiguated_orgs.append("United Nations")
        elif (org == "EC" or org == "the Election Commission"):
            disambiguated_orgs.append("Election Commission")
        elif (org == "the Supreme Court"):
            disambiguated_orgs.append("Supreme Court")
        elif (org == "the White House"):
            disambiguated_orgs.append("White House")
        elif (org == "Jamaat"):
            disambiguated_orgs.append("Jamaat-e-Islami")
        else:
            disambiguated_orgs.append(org)
    return (list(set(disambiguated_orgs)))

In [65]:
# df['news_ner_tags']['persons_unique'] = df['news_ner_tags']['persons_unique'].apply(name_disambiguation)

In [66]:
ner = df['news_ner_tags'].apply(pd.Series)

In [69]:
ner.columns

Index(['dates', 'dates_unique', 'locations', 'locations_unique', 'moneys',
       'moneys_unique', 'organizations', 'organizations_unique', 'percents',
       'percents_unique', 'persons', 'persons_unique', 'times',
       'times_unique'],
      dtype='object')

In [68]:
df.shape

(49055, 17)

In [70]:
ner = ner.drop(['dates', 'dates_unique', 'locations', 'moneys',
       'moneys_unique', 'organizations', 'percents',
       'percents_unique', 'persons', 'times',
       'times_unique'], axis=1)

In [72]:
ner.shape

(49055, 3)

In [73]:
df = df.drop(['news_ner_tags'], axis=1)

In [75]:
df = pd.concat([df, ner], axis=1)

In [76]:
df.shape

(49055, 19)

In [82]:
df.columns

Index(['_id', 'is_negative', 'news_crawled_date', 'news_headline',
       'news_image_urls', 'news_keywords', 'news_location', 'news_ml_tags',
       'news_naive_tags', 'news_original_tags', 'news_publish_date',
       'news_reporters', 'news_text', 'news_url', 'newspaper_name',
       'newspaper_url', 'locations_unique', 'organizations_unique',
       'persons_unique'],
      dtype='object')

In [86]:
df['persons_unique'] = df['persons_unique'].apply(name_disambiguation)
df['organizations_unique'] = df['organizations_unique'].apply(org_disambiguation)

In [89]:
# df.to_pickle('Data/DT/DT.pkl')

In [8]:
# def check_alleged(text):
#     if re.search(r'\balleged\b', text):
#         return True
#     return False

# conversational_words = ['said', 'told', 'asked', 'speak', 'say', 'tell', 'spoke', 'added', 'declare']

In [9]:
# from pprint import pprint
# a = tokenizer.tokenize(df['news_text'][0])
# for i in a:
#     i = i.replace("\n"," ")
#     print(i+"\n")

In [10]:
# for index, row in df.iterrows():
# #     print(row['_id'], index)
#     period_delimited_list = tokenizer.tokenize(row['news_text'])
#     for sentence in period_delimited_list:
#         if any(word in sentence for word in conversational_words) or check_alleged(sentence):
            
    
    

In [11]:
# df['news_text'][0]